# Data Analysis and Visualization in Python
## Combining DataFrames with pandas
Questions
* Can I work with data from multiple sources?
* How can I combine data from different data sets?

Objectives
* Combine data from multiple files into a single DataFrame using merge and concat.
* Combine two DataFrames using a unique ID found in both DataFrames.
* Employ `to_csv` to export a DataFrame in CSV format.
* Join DataFrames using common fields (join keys).

## Loading our data

In [ ]:
# first make sure pandas is loaded
import pandas as pd

In [ ]:
# read in the survey csv
surveys_df = pd.read_csv("../data/surveys.csv", keep_default_na=False, na_values=[""])
surveys_df.head()

In [ ]:
# read in the species csv
species_df = pd.read_csv("../data/species.csv", keep_default_na=False, na_values=[""])
species_df.head()

## Concatenating DataFrames

In [ ]:
# read in first 10 lines of surveys table
survey_sub = surveys_df.head(10)
survey_sub

In [ ]:
# grab the last 10 rows (minus the last one)
survey_sub_last10 = surveys_df.tail(10)
survey_sub_last10

In [ ]:
#reset the index values to the second dataframe appends properly
# drop=True option avoids adding new index column with old index values
survey_sub_last10 = survey_sub_last10.reset_index(drop=True)
survey_sub_last10

In [ ]:
# stack the DataFrames on top of each other
vertical_stack = pd.concat([survey_sub, survey_sub_last10], axis=0)
vertical_stack # You may want to reset the index again

In [ ]:
# place the DataFrames side by side
horizontal_stack = pd.concat([survey_sub, survey_sub_last10], axis=1)
horizontal_stack

### Writing Out Data to CSV

In [ ]:
# Write DataFrame to CSV
vertical_stack.to_csv('out.csv', index=False)

In [ ]:
# For kicks read our output back into python and make sure all looks good
new_output = pd.read_csv('out.csv', keep_default_na=False, na_values=[""])
new_output

## Exercise - Concatenating DataFrames
In the `data/yearly_files` folder, there are two survey data files: `surveys2001.csv` and `surveys2002.csv`. Read the data into python and combine the files to make one new data frame. Create a plot of average plot weight by year grouped by sex. Export your results as a CSV and make sure it reads back into python properly.

In [ ]:
# read the files:
survey2001 = pd.read_csv("../data/yearly_files/surveys2001.csv")
survey2002 = pd.read_csv("../data/yearly_files/surveys2002.csv")
# concatenate
survey_all = pd.concat([survey2001, survey2002], axis=0)

In [ ]:
# get the weight for each year, grouped by sex:
weight_year = survey_all.groupby(['year', 'sex'])["weight"].mean().unstack()
weight_year

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# plot:
weight_year.plot(kind="bar")
plt.tight_layout()  # tip(!)

In [ ]:
# writing to file:
weight_year.to_csv("weight_for_year.csv")

# reading it back in:
pd.read_csv("weight_for_year.csv", index_col=0)

## Joining DataFrames
### Joining Two DataFrames

In [ ]:
# read in first 10 lines of surveys table
survey_sub = surveys_df.head(10)
survey_sub

In [ ]:
# import a small subset of the species data designed for this part of the lesson
species_sub = pd.read_csv('../data/speciesSubset.csv', keep_default_na=False, na_values=[""])
species_sub

### Identifying join keys

In [ ]:
survey_sub.columns

In [ ]:
species_sub.columns

### Inner joins

![Inner join of tables A and B](https://datacarpentry.org/python-ecology-lesson/fig/inner-join.png)

In [ ]:
merged_inner = pd.merge(left=survey_sub, right=species_sub,
                        left_on='species_id', right_on='species_id')
# what's the size of the output data?
merged_inner.shape

In [ ]:
merged_inner

### Left joins

![Left join of tables A and B](https://datacarpentry.org/python-ecology-lesson/fig/left-join.png)

In [ ]:
merged_left = pd.merge(left=survey_sub, right=species_sub, how='left',
                       left_on='species_id', right_on='species_id')
# what's the size of the output data?
merged_left.shape

In [ ]:
merged_left

In [ ]:
merged_left[ pd.isnull(merged_left.genus) ]

### Other join types
* `how='right'` : all rows from the right DataFrame are kept
* `how='outer'` : all pairwise combinations of rows from both DataFrames

## Exercise - Joining all data
Create a new DataFrame by joining the contents of the `surveys.csv` and `species.csv` tables. Then calculate and plot the distribution of:
1. taxa by site (number of species of each `taxa` per site "`plot_id`")
1. taxa by sex by site

In [ ]:
merged_left = pd.merge(left=surveys_df, right=species_df, how='left', on="species_id")

In [ ]:
merged_left.groupby(["plot_id"])["taxa"].nunique().plot(kind='bar')

In [ ]:
plot_taxa_plot = merged_left.groupby(["plot_id", "taxa"])["record_id"].count().unstack()
plot_taxa_plot

In [ ]:
plot_taxa_plot.plot(kind='bar', stacked=True)
plt.legend(loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.05))

In [ ]:
# Part 2
merged_left.loc[merged_left["sex"].isnull(), "sex"] = 'M|F'
merged_left.loc[~merged_left["sex"].isin(['F', 'M', 'M|F']), "sex"] = 'M|F'

In [ ]:
ntaxa_sex_plot = merged_left.groupby(["plot_id", "sex"])["taxa"].nunique().reset_index(level=1)
ntaxa_sex_plot

In [ ]:
# Use pivot_table() instead of unstack()
ntaxa_sex_plot = ntaxa_sex_plot.pivot_table(values="taxa", columns="sex", index=ntaxa_sex_plot.index)
ntaxa_sex_plot

In [ ]:
ntaxa_sex_plot.plot(kind="bar", stacked=True, legend=False)
plt.legend(loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.08),
           fontsize='small', frameon=False)